------------------
# Extraction

In [261]:
import pyspark
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('customer_etl').getOrCreate()

Let's load cdw_sapp_custmer.json

In [262]:
cust_df = spark.read.json("source_data/cdw_sapp_custmer.json")

--------------------------------
# Exploratory Analysis & Tranforming

In [263]:
cust_df.printSchema()
cust_df.show(10)

root
 |-- APT_NO: string (nullable = true)
 |-- CREDIT_CARD_NO: string (nullable = true)
 |-- CUST_CITY: string (nullable = true)
 |-- CUST_COUNTRY: string (nullable = true)
 |-- CUST_EMAIL: string (nullable = true)
 |-- CUST_PHONE: long (nullable = true)
 |-- CUST_STATE: string (nullable = true)
 |-- CUST_ZIP: string (nullable = true)
 |-- FIRST_NAME: string (nullable = true)
 |-- LAST_NAME: string (nullable = true)
 |-- LAST_UPDATED: string (nullable = true)
 |-- MIDDLE_NAME: string (nullable = true)
 |-- SSN: long (nullable = true)
 |-- STREET_NAME: string (nullable = true)

+------+----------------+------------+-------------+--------------------+----------+----------+--------+----------+---------+--------------------+-----------+---------+-----------------+
|APT_NO|  CREDIT_CARD_NO|   CUST_CITY| CUST_COUNTRY|          CUST_EMAIL|CUST_PHONE|CUST_STATE|CUST_ZIP|FIRST_NAME|LAST_NAME|        LAST_UPDATED|MIDDLE_NAME|      SSN|      STREET_NAME|
+------+----------------+------------+---

How many rows do we have in total in this dataframe?

In [264]:
cust_df.count()

952

Let's rearrange the columns so they make a bit more sense when looking at them.

In [265]:
cust_df = cust_df.select('CREDIT_CARD_NO', 'SSN', 'CUST_EMAIL', 'CUST_PHONE',\
            'FIRST_NAME','MIDDLE_NAME', 'LAST_NAME', 'APT_NO', 'STREET_NAME',\
            'CUST_CITY', 'CUST_ZIP', 'CUST_STATE', 'CUST_COUNTRY', 'LAST_UPDATED')

cust_df.show(5)

+----------------+---------+-------------------+----------+----------+-----------+---------+------+-----------------+------------+--------+----------+-------------+--------------------+
|  CREDIT_CARD_NO|      SSN|         CUST_EMAIL|CUST_PHONE|FIRST_NAME|MIDDLE_NAME|LAST_NAME|APT_NO|      STREET_NAME|   CUST_CITY|CUST_ZIP|CUST_STATE| CUST_COUNTRY|        LAST_UPDATED|
+----------------+---------+-------------------+----------+----------+-----------+---------+------+-----------------+------------+--------+----------+-------------+--------------------+
|4210653310061055|123456100|AHooper@example.com|   1237818|      Alec|         Wm|   Hooper|   656|Main Street North|     Natchez|   39120|        MS|United States|2018-04-21T12:49:...|
|4210653310102868|123453023|EHolman@example.com|   1238933|      Etta|    Brendan|   Holman|   829|    Redwood Drive|Wethersfield|   06109|        CT|United States|2018-04-21T12:49:...|
|4210653310116272|123454487|WDunham@example.com|   1243018|    Wilber|

In [266]:
cust_df.describe().show()

+-------+--------------------+--------------------+--------------------+------------------+----------+-----------+---------+------------------+-----------+---------+------------------+----------+-------------+--------------------+
|summary|      CREDIT_CARD_NO|                 SSN|          CUST_EMAIL|        CUST_PHONE|FIRST_NAME|MIDDLE_NAME|LAST_NAME|            APT_NO|STREET_NAME|CUST_CITY|          CUST_ZIP|CUST_STATE| CUST_COUNTRY|        LAST_UPDATED|
+-------+--------------------+--------------------+--------------------+------------------+----------+-----------+---------+------------------+-----------+---------+------------------+----------+-------------+--------------------+
|  count|                 952|                 952|                 952|               952|       952|        952|      952|               952|        952|      952|               952|       952|          952|                 952|
|   mean|4.210653353718597...|1.2345552588130252E8|                null|1239

From the above summary we can see that the credit card numbers are all 16 digits long, SSN numbers are all 9 digits long, zip codes are all 5 digits long, and all the customers are from the United States.  The only issue is that the phone numbers are only 7 digits long but should be 10 digits. 

It seems like the CUST_ZIP and CUST_STATE columns are in string format so the min and max values might only consider the first character when considering order of values.  Lets confirm that the states are 2 characters in length and the zip codes are all 5 digits long.

In [267]:
cust_df.withColumn("zip_len", F.length(cust_df["CUST_ZIP"]))\
    .groupBy("zip_len").count().show()

cust_df.withColumn("state_len", F.length(cust_df["CUST_STATE"]))\
    .groupBy("state_len").count().show()

+-------+-----+
|zip_len|count|
+-------+-----+
|      5|  952|
+-------+-----+

+---------+-----+
|state_len|count|
+---------+-----+
|        2|  952|
+---------+-----+



Lets make sure the emails are all valid. All have a "@" and a "." (for example, ".com") and no blank spaces.

In [268]:
email_filter_df = cust_df.filter(~F.col('CUST_EMAIL').rlike('^\S+@\S+\.\S+$'))
email_filter_df.select('CUST_EMAIL').show()

#^\S+@\S+\.\S+$ checks to see if the string is a valid email address.  the "^\S+" checks
#if there are one or more non-whitespace characters at the start of the string.  Can't have
#blank spaces in an email address. The ^ is necessary because "ex ample@example.com" would be
#a match on "ample@example.com".  We check for "@" with one or more non-whitespace character after it
#followed by a "." (\. needs to be used because just a "." in regex matches to any single character).
#Finished off by "\S+$" which matches one or more non-whitespace characters that end the string.  
#The "$" is necessary for the same reason that the "^" is necessary.  The "~" is like a NOT so only
#the non matches will be shown.

+----------+
|CUST_EMAIL|
+----------+
+----------+



The customer cities look bunched up, aka two word cities have no spaces between the words.  Let's fix that.

In [269]:
cust_df = cust_df.withColumn('CUST_CITY', 
        F.regexp_replace(cust_df['CUST_CITY'], "(?<=.)([A-Z])", ' $1'))
cust_df.select('CUST_CITY').show(5)

#(?<=.) is a positive lookbehind.  It checks if there is any character behind ([A-Z]).  Don't
#want to match the capital letters in the beginning of the string.
#([A-Z]) looks for any capital letter.  In parantheses because we want to capture this group.
#' $1' Blank space plus the first, and only, captured group. aka this returns the matched 
#capital letter, otherwise the matched capital letter would have been replaced.

+------------+
|   CUST_CITY|
+------------+
|     Natchez|
|Wethersfield|
|     Huntley|
|  New Berlin|
|     El Paso|
+------------+
only showing top 5 rows



Let's make sure all the CC numbers, SSNs, Phone numbers, and Emails are unique. Remember, there are 952 total entries.

In [270]:
cust_df.select(F.countDistinct("CREDIT_CARD_NO")).show()
cust_df.select(F.countDistinct("SSN")).show()
cust_df.select(F.countDistinct("CUST_PHONE")).show()

+------------------------------+
|count(DISTINCT CREDIT_CARD_NO)|
+------------------------------+
|                           952|
+------------------------------+

+-------------------+
|count(DISTINCT SSN)|
+-------------------+
|                952|
+-------------------+

+--------------------------+
|count(DISTINCT CUST_PHONE)|
+--------------------------+
|                       901|
+--------------------------+



Hmmm, the Phone numbers don't seem to be unique.  Let's explore further.

In [271]:
cust_df.groupBy('CUST_PHONE').count().orderBy(F.col('count').desc()).show(10)

+----------+-----+
|CUST_PHONE|count|
+----------+-----+
|   1241898|    3|
|   1236886|    3|
|   1237294|    3|
|   1240382|    2|
|   1243459|    2|
|   1239063|    2|
|   1242677|    2|
|   1235508|    2|
|   1242999|    2|
|   1239668|    2|
+----------+-----+
only showing top 10 rows



Let's look at a sample phone number to see if any of the other customer info overlaps.

In [272]:
cust_df.where(cust_df['CUST_PHONE'] == '1236886').show()

+----------------+---------+--------------------+----------+----------+-----------+---------+------+--------------+---------+--------+----------+-------------+--------------------+
|  CREDIT_CARD_NO|      SSN|          CUST_EMAIL|CUST_PHONE|FIRST_NAME|MIDDLE_NAME|LAST_NAME|APT_NO|   STREET_NAME|CUST_CITY|CUST_ZIP|CUST_STATE| CUST_COUNTRY|        LAST_UPDATED|
+----------------+---------+--------------------+----------+----------+-----------+---------+------+--------------+---------+--------+----------+-------------+--------------------+
|4210653315752398|123455638|FMeredith@example...|   1236886|   Francis|    Donnell| Meredith|   417|Lincoln Street| Wilmette|   60091|        IL|United States|2018-04-21T12:49:...|
|4210653352401004|123456941|  EWells@example.com|   1236886|     Edwin|      Alice|    Wells|   890|    5th Avenue|Mundelein|   60060|        IL|United States|2018-04-21T12:49:...|
|4210653399859149|123454047| EBeatty@example.com|   1236886|     Emery|    Susanna|   Beatty|  

In this sample it seems like 3 different people have the same number. I would consider using an area code that corresponds to the customer's location but in this example two of the customers are in the same state and fall under the same area code.

Let's add an area code to the phone numbers based on the customer's zip code and see if that helps differentiate the data.  First, lets load the area code data into a dataframe.  I cleaned up and created this file in the "area_code_data" notebook file.

In [273]:
area_code_df = spark.read.json("source_data/area_codes.json")
area_code_df.show(5)

+---------+----------+-------+
|     city|  npa_list|zipCode|
+---------+----------+-------+
| Adjuntas|[939, 787]|  00601|
|   Aguada|     [787]|  00602|
|Aguadilla|     [787]|  00603|
|  Maricao|[939, 787]|  00606|
|   Anasco|     [787]|  00610|
+---------+----------+-------+
only showing top 5 rows



Since there can be multiple NPA numbers (area codes) for some individual zip codes and there are multiple customers from the same zip code, I decided to alternate the distribution of area codes.  When I would take an area code from the list in the npa_list column, I would move it to the back of the list so that I could just pull the first value in that list for every subsequent query.

To achieve this, I first converted the area_code_df to a pandas dataframe.  I tried to implement this in a spark dataframe at first but it was running way too slow.  The reason being, in order to update the list in the npa_list column of the spark df, I had to update the entire column and dataframe, even though I just needed to update one cell.  Pandas allows for single cell updates and at a much faster speed.  The spark dataframe took 20 minutes and was only 80% completed before it got hung up indefinitely.  The same task in pandas only took 7 seconds to complete.

In the below code I am interating through each row of the cust_df spark dataframe and looking at the zip code in that row.  Then I am looking up the zipcode in the pandas dataframe.  If it finds a match then I am adding the first npa from the npa_list list to a running list called area_code_list.  This list will later get added to the spark dataframe which had to be done this way because you can change a row value in a column one row at a time, in spark dfs you have to update the entire dataframe every time you want to make a change to it.  Then I place the first npa number to the end of the list and save that reordered list to the pandas df using this line: ```npa_list.append(npa_list.pop(0))```

There is also an issue where some users in the customer data have a zip code that doesn't exist in the area_code dataset.  I assigned a ```None``` value to those users for now.

In [274]:
area_code_pd = area_code_df.toPandas()

# This empty list will contain the chosen NPA number for each row.  It will later be joined to 
# the spark dataframe as a new column.
area_code_list = []

# We are taking each row of the customer dataframe and extracting the zipcode and city for the 
# chosen customer.  
for row in cust_df.collect():  #have to use collect.  collect() returns a list of rows.  you can 
    # iterate over this list.  if you don't use the collect() it will return a column object which 
    # can't be iterated over.
    
    zip_code = row[10]
    city = row[9]

    # Here we are saving the npa_list list into a npa_list variable and then taking the first 
    # object in that list and appending it to the running list we created before the for loop.
    # Since there are some customer's with zip codes that don't match the area_code dataframe
    # we try to find a matching NPA number based on their city.
    try:
        npa_list = area_code_pd.loc[area_code_pd['zipCode'] == zip_code, 'npa_list'].iloc[0]
        area_code_list.append(npa_list[0])
        index_pos = area_code_pd[area_code_pd['zipCode'] == zip_code].index[0]
    except:
        npa_list = area_code_pd.loc[area_code_pd['city'] == city, 'npa_list'].iloc[0]
        area_code_list.append(npa_list[0])
        index_pos = area_code_pd[area_code_pd['city'] == city].index[0]

    # Below two lines reorders the python list of npa nums so that the first num goes to the back
    # of the list.  The next line assigns this reordered list to the position of the original
    # npa list in the pandas dataframe.
    npa_list.append(npa_list.pop(0))
    area_code_pd.at[index_pos, 'npa_list'] = npa_list

Let's make sure the list is the same length as the customer dataframe (952) so there aren't any errors when we add it as a column.

In [275]:
len(area_code_list)

952

Let's join the list of NPA numbers to the customer dataframe.

In [276]:
#had to use pandas because adding a list in pandas is super easy.  in pyspark I would have to 
#create a dataframe for the python list and add an incrementing index column for both dataframes 
#and then join them on the index column and then drop that column.

pandas_cust = cust_df.toPandas()
pandas_cust['npa'] = area_code_list
cust_df = spark.createDataFrame(pandas_cust)
cust_df.show(10)

+----------------+---------+--------------------+----------+----------+-----------+---------+------+-----------------+-------------+--------+----------+-------------+--------------------+---+
|  CREDIT_CARD_NO|      SSN|          CUST_EMAIL|CUST_PHONE|FIRST_NAME|MIDDLE_NAME|LAST_NAME|APT_NO|      STREET_NAME|    CUST_CITY|CUST_ZIP|CUST_STATE| CUST_COUNTRY|        LAST_UPDATED|npa|
+----------------+---------+--------------------+----------+----------+-----------+---------+------+-----------------+-------------+--------+----------+-------------+--------------------+---+
|4210653310061055|123456100| AHooper@example.com|   1237818|      Alec|         Wm|   Hooper|   656|Main Street North|      Natchez|   39120|        MS|United States|2018-04-21T12:49:...|601|
|4210653310102868|123453023| EHolman@example.com|   1238933|      Etta|    Brendan|   Holman|   829|    Redwood Drive| Wethersfield|   06109|        CT|United States|2018-04-21T12:49:...|860|
|4210653310116272|123454487| WDunham@exa

Let's look at a sample zip code and see if the NPA numbers were assigned in an alternating fashion.

In [277]:
cust_df.select("CUST_ZIP","FIRST_NAME","npa").where(cust_df['CUST_ZIP'] == "01810").show()

+--------+----------+---+
|CUST_ZIP|FIRST_NAME|npa|
+--------+----------+---+
|   01810|   Janelle|781|
|   01810|   Pearlie|978|
|   01810|    Prince|617|
|   01810|     Elmer|781|
|   01810|     Dusty|978|
|   01810|   Allison|617|
|   01810|   Liliana|781|
|   01810|     Tanya|978|
|   01810|  Isabella|617|
|   01810|  Courtney|781|
|   01810|  Mckinley|978|
|   01810|   Jacinto|617|
+--------+----------+---+



Now let's see how many duplicate numbers there are.

In [278]:
cust_df.groupBy('CUST_PHONE','npa').count().orderBy(F.col('count').desc()).show(10)

+----------+---+-----+
|CUST_PHONE|npa|count|
+----------+---+-----+
|   1238106|412|    2|
|   1240339|310|    1|
|   1241408|262|    1|
|   1241213|310|    1|
|   1239268|406|    1|
|   1236077|469|    1|
|   1242570|440|    1|
|   1238165|915|    1|
|   1236561|770|    1|
|   1240689|517|    1|
+----------+---+-----+
only showing top 10 rows



A big improvement!  Let's investigate further to see if we can fix the last duplicate phone number.  Let's look at the zip codes for the two 1238106 numbers and then lets see if there is an alternative npa available for that zip code.

In [279]:
cust_df.where(cust_df['CUST_PHONE'] == '1238106').show()
area_code_pd.loc[area_code_pd['zipCode'] == "15317", 'npa_list'].iloc[0]

+----------------+---------+-------------------+----------+----------+-----------+---------+------+---------------+-----------+--------+----------+-------------+--------------------+---+
|  CREDIT_CARD_NO|      SSN|         CUST_EMAIL|CUST_PHONE|FIRST_NAME|MIDDLE_NAME|LAST_NAME|APT_NO|    STREET_NAME|  CUST_CITY|CUST_ZIP|CUST_STATE| CUST_COUNTRY|        LAST_UPDATED|npa|
+----------------+---------+-------------------+----------+----------+-----------+---------+------+---------------+-----------+--------+----------+-------------+--------------------+---+
|4210653347791178|123458703|RHorton@example.com|   1238106|      Russ|   Terrance|   Horton|    94| Carriage Drive| Canonsburg|   15317|        PA|United States|2018-04-21T12:49:...|412|
|4210653376655865|123454594|CMedina@example.com|   1238106|     Carly|     Celina|   Medina|    76|Hillside Avenue|Monroeville|   15146|        PA|United States|2018-04-21T12:49:...|412|
+----------------+---------+-------------------+----------+------

['724', '412']

Looks like '724' is available as an alternative.  Let's change one of the numbers to use this NPA number.

In [280]:
cust_df = cust_df.withColumn('npa',
            F.when((F.col("CUST_ZIP") == "15317") & (F.col("CUST_PHONE") == "1238106"), "724" )\
                .otherwise(F.col('npa')))

In [281]:
cust_df.groupBy('CUST_PHONE','npa').count().orderBy(F.col('count').desc()).show(5)

+----------+---+-----+
|CUST_PHONE|npa|count|
+----------+---+-----+
|   1240339|310|    1|
|   1241408|262|    1|
|   1241213|310|    1|
|   1239268|406|    1|
|   1236077|469|    1|
+----------+---+-----+
only showing top 5 rows



Nice! No more duplicate phone numbers.

Let's change the phone number to a (XXX)XXX-XXXX format.

In [282]:
cust_df = cust_df.withColumn('CUST_PHONE',
            F.format_string("(%s)%s-%s", cust_df['npa'],
            cust_df['CUST_PHONE'][0:3], cust_df['CUST_PHONE'][4:4]))

In [283]:
cust_df.select('CUST_PHONE').show(5)

+-------------+
|   CUST_PHONE|
+-------------+
|(601)123-7818|
|(860)123-8933|
|(224)124-3018|
|(262)124-3215|
|(915)124-2074|
+-------------+
only showing top 5 rows



Let's look at the emails column to see if they are all unique.

In [284]:
cust_df.select(F.countDistinct("CUST_EMAIL")).show()

+--------------------------+
|count(DISTINCT CUST_EMAIL)|
+--------------------------+
|                       928|
+--------------------------+



Strange, why are there so many duplicate emails?

In [285]:
cust_df.groupBy('CUST_EMAIL').count().orderBy(F.col('count').desc()).show(10)

+--------------------+-----+
|          CUST_EMAIL|count|
+--------------------+-----+
| ETruong@example.com|    2|
| SCrouch@example.com|    2|
|   BYork@example.com|    2|
| DDorsey@example.com|    2|
|  JEmery@example.com|    2|
|   RHood@example.com|    2|
|  MHatch@example.com|    2|
|JGodfrey@example.com|    2|
| RDeleon@example.com|    2|
|SPadgett@example.com|    2|
+--------------------+-----+
only showing top 10 rows



Let's look at a sample email address and explore further.

In [286]:
cust_df.select('SSN','CUST_EMAIL','CUST_PHONE','FIRST_NAME','MIDDLE_NAME',).where(cust_df['CUST_EMAIL'] == 'BYork@example.com').show()

+---------+-----------------+-------------+----------+-----------+
|      SSN|       CUST_EMAIL|   CUST_PHONE|FIRST_NAME|MIDDLE_NAME|
+---------+-----------------+-------------+----------+-----------+
|123452454|BYork@example.com|(561)123-5377|    Bertie|   Mercedes|
|123458062|BYork@example.com|(609)123-5585|    Buster|    Kirsten|
+---------+-----------------+-------------+----------+-----------+



Conclusion: I really shouldn't alter emails since the email communication will not function if the email address isn't 100% correct.  

But since two seemingly different people seem to be sharing the same email I can only assume the emails are made up and aren't actually used for communication.  

So, as an exercise in data transformation, lets try to alter the emails so we have less unique email addresses.  My solution:  The emails seems to corresspond to the customer's first initial and last names.  But these combinations seems to overlap so lets differentiate the duplicate combinations by adding in a middle initial to the email address.

First, we will use regex to extract the first letter of the customer's middle name and store it in a new column.

In [287]:
cust_df = cust_df.withColumn('Middle_Initial',F.format_string("%s",cust_df['MIDDLE_NAME'][0:1]))

Let's see how this new column looks and if it matches the middle name.

In [288]:
cust_df.select('MIDDLE_NAME','Middle_Initial').show(5)

+-----------+--------------+
|MIDDLE_NAME|Middle_Initial|
+-----------+--------------+
|         Wm|             W|
|    Brendan|             B|
|   Ezequiel|             E|
|      Trina|             T|
|        May|             M|
+-----------+--------------+
only showing top 5 rows



Let's insert the Middle Initial into the second position of the email address by using the format_string function.  

In [289]:
cust_df = cust_df.withColumn('CUST_EMAIL', F.format_string("%s%s%s", cust_df['CUST_EMAIL'][1:1],
            cust_df['Middle_Initial'], cust_df['CUST_EMAIL'][2:30]))

In [291]:
cust_df.select('CUST_EMAIL','FIRST_NAME','MIDDLE_NAME','LAST_NAME').show(5)

+--------------------+----------+-----------+---------+
|          CUST_EMAIL|FIRST_NAME|MIDDLE_NAME|LAST_NAME|
+--------------------+----------+-----------+---------+
|AWHooper@example.com|      Alec|         Wm|   Hooper|
|EBHolman@example.com|      Etta|    Brendan|   Holman|
|WEDunham@example.com|    Wilber|   Ezequiel|   Dunham|
| ETHardy@example.com|   Eugenio|      Trina|    Hardy|
| WMAyers@example.com|   Wilfred|        May|    Ayers|
+--------------------+----------+-----------+---------+
only showing top 5 rows



Let's see how many duplicate emails there are now.  

In [290]:
cust_df.select(F.countDistinct("CUST_EMAIL")).show()
cust_df.groupBy('CUST_EMAIL').count().orderBy(F.col('count').desc()).show(5)

+--------------------------+
|count(DISTINCT CUST_EMAIL)|
+--------------------------+
|                       951|
+--------------------------+

+--------------------+-----+
|          CUST_EMAIL|count|
+--------------------+-----+
|KCBonner@example.com|    2|
|LEGaines@example.com|    1|
|AWHooper@example.com|    1|
|HHMckinney@exampl...|    1|
| JRMoody@example.com|    1|
+--------------------+-----+
only showing top 5 rows



There seems to be one left.  We could eliminate the redudancy further by adding the second letter of the first or middle name into the email address but I will leave it as is for now.

The next requirement is for the First and Last Names to be in Title Case, aka the first letter needs to be capitalized.  Let's use regex to see if that is the case in our data.

In [293]:
f_name_filter = cust_df.filter(~F.col('FIRST_NAME').rlike('^[A-Z][a-z]+$'))
l_name_filter = cust_df.filter(~F.col('LAST_NAME').rlike('^[A-Z][a-z]+$'))

# ^[A-Z] checks to see if there is a single capital letter at the start of the string.
# [a-z]+$ checks to see if there are only lower case letters after the first letter 
# and those letters also go on till the end of the string.  This eliminates the 
# possible matches of McCourt since the C cuts off the lower case letters.

f_name_filter.select('FIRST_NAME').show()
l_name_filter.select('LAST_NAME').show()

+----------+
|FIRST_NAME|
+----------+
+----------+

+---------+
|LAST_NAME|
+---------+
+---------+



Next, we need to convert the middle names to lower case.  First let's see if we have any names that need to be converted.